In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import statsmodels as sm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
nottem = pd.read_csv("../input/nottingham-temp/nottem.csv",header=0,index_col=0,names=['index','temp'])

In [ ]:
lynx = pd.read_csv("../input/lynxdataset/LYNXdata.csv",header=0,index_col=0)

In [ ]:
lynxseries=pd.Series(lynx['trappings'].values,index=pd.DatetimeIndex(data=(tuple(pd.date_range('31/12/1821',
                                                                                               periods=114,
                                                                                               freq='A-DEC'))),
                                                                    freq='A-DEC'))

In [ ]:
lynxseries.head()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
def statistics_testing(timeseries):
    from statsmodels.tsa.stattools import adfuller
    print('Test Results')
    test=adfuller(timeseries,autolag='AIC')
    test_res=pd.Series(test[0:4],index=['Test Statistic','p-value','Number of Lags','Number of Observations Used'])
    print(test_res)

In [ ]:
statistics_testing(lynxseries)

In [ ]:
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(12,8))
plot_acf(lynxseries,lags=20,ax=ax1)
plot_pacf(lynxseries,lags=20,ax=ax2)
ax1.set_title('ACF plot for lynx')
ax2.set_title('PACF plot for lynx')

In [ ]:
model=ARIMA(lynxseries,order=(4,0,0))
result_AR4=model.fit()
plt.figure(figsize=(12,8))
plt.plot(lynxseries)
plt.plot(result_AR4.fittedvalues,color='red')

In [ ]:
lynxseries.tail()

In [ ]:
result_AR4.fittedvalues.tail()

In [ ]:
result_AR4.resid.tail()

In [ ]:
np.mean(result_AR4.resid)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = plot_acf(result_AR4.resid, lags=20, ax=ax1)

In [ ]:
def resid_histogram(data):
    from scipy.stats import norm
    
    plt.figure(figsize=(10,6))
    plt.hist(data,bins='auto',density=True,rwidth=0.85,label='Residuals')
    mean_resid,std_resid=norm.fit(data)
    xmin,xmax=plt.xlim()
    curve_length=np.linspace(xmin,xmax,100)
    bell_curve=norm.pdf(curve_length,mean_resid,std_resid)
    plt.plot(curve_length,bell_curve,'m',linewidth=2)
    plt.grid(axis='y',alpha=0.2)
    plt.xlabel('Residuals')
    plt.ylabel('Density')
    plt.title('Residual vs Normal distribution - Mean={0} Std={1}'.format(round(mean_resid,2),round(std_resid,2)))
    plt.show()

In [ ]:
resid_histogram(result_AR4.resid)

In [ ]:
result_AR4.summary()

In [ ]:
fcast400=result_AR4.predict(start='31/12/1935',end='31/12/1945')

In [ ]:
model202=ARIMA(lynxseries,order=(2,0,2))
result_AR202=model202.fit()
fcast202=result_AR202.predict(start='31/12/1935',end='31/12/1945')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(lynxseries,linewidth=2,label='Original')
plt.plot(fcast400,color='red',linewidth=2,label='ARIMA 400')
plt.plot(fcast202,color='black',linewidth=2,label='ARIMA 202')
plt.legend()

**SARIMA MODEL**

In [ ]:
nottem_df=pd.DataFrame((nottem.temp).values,columns=['temperature'],index=pd.date_range('1920-01-31',periods=240,freq='M'))
nottemseries=pd.Series((nottem.temp).values,index=pd.date_range('1920-01-31',periods=240,freq='M'))

In [ ]:
nottem_df.head()

In [ ]:
from statsmodels.graphics.tsaplots import month_plot
fig,ax1=plt.subplots(1,1,figsize=(12,8))
month_plot(nottemseries,ax=ax1)
plt.title('Month plot nottem dataset')
plt.grid(axis='both')
plt.tight_layout()

In [ ]:
pivot_df=pd.pivot_table(nottem_df,index=nottem_df.index.month,columns=nottem_df.index.year,values='temperature')
pivot_df

In [ ]:
month_names=('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
pivot_df.index=month_names
pivot_df

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(pivot_df)
plt.grid(axis='both')
plt.legend(pivot_df.columns)
plt.tight_layout()

In [ ]:
!pip install pmdarima

In [ ]:
import pmdarima as pm

In [ ]:
sarima=pm.auto_arima(nottemseries,error_action='ignore',suppress_warnings=True,seasonal=True,m=12,star_p=1,start_q=1,start_P=0,start_Q=0,D=1,max_order=5,max_d=1,max_D=1,stepwise=False,trace=True)

In [ ]:
sarima.summary()